## Machine Learning Summer School 2021 - Reinforcement Learning Workshop
# Assignment 1: Q-Learning

Welcome to Q-Learning part of the workshop! :) Here you will be tasked with implementing the Q-Learning algorithm that was covered by lectures. This notebook is divided into three main sections: 

1. **Problem description**: Where we introduce the problem we are going to try to solve using Q-Learning.
2. **Q-Learning implementation**: Where we provide the basic structure of the Q-Learning algorithm and ask you to fill in all important steps.
3. **Experimentation**: Where we encourage you to play around with hyper-parameters a bit, and discuss the resulting solutions.

**NOTE!**: Certain portion of the code is hidden, and only imported into this notebook. Namely we have hidden the **Ice** environment, which will be used for demonstration of the Q-Learning algorithm. You can find it [here](../edit/ice_env.py).

## Problem description

The ice world is a small, 4x4 icy grid. A robot is tasked to cross through the ice to reach the opposite side of the small world, while, if possible, collecting some treasure from a shipwreck embedded in the ice.
Unfortunately, the ice has been melting, is slippery, and is already cracked in several places. The robot must avoid falling through the cracks! Due to the slippery nature of the ice, the robot risks sliding on it at each step.

We visualize the world in the image below. Agent always starts from the bottom left corner, where you will see it waving to you. The goal location is in the upper right corner, represented by a flag. Shipwreck location is indicated by a sinking ship, while the rest of the blocks are either ice or cracks. We further denote axis which will be used when constructing the Q-value table -- indexing starts at zero, $x$ iterates over columns, while $y$ iterates over rows. Q-value table is, thus, indexed by $(y,x)$. 

<figure class="image">
  <img src="./resources/ice_world.png" alt="Ice World environment." width="400">
</figure>

The robot tries to take one step on the ice, one cell at a time, vertically or horizontally (not diagonally). At each time-step, the robot has a 5% chance of slipping on the ice, and go all the way to the side of the environment. For instance, if the robot tries to go up from any cell in the first column, it may slide across the environment and end up on the top-left cell (or in any crack it may encounter while sliding). The robot cannot move outside the environment, so trying to go up when in the first row has no effect.
Reaching the goal rewards the robot with 100 points. Passing on the shipwreck will allow the robot to collect some of the treasure inside: each time the robot passes there it will get an additional 20 points reward. If the robot falls through the cracks is destroyed (ending the episode), and so it's penalized with -10 points. Additionally it has been added that each ice step is penalized with -5 points in order to motivate the robot to finish the game as soon as possible.

More formally, the environment has a total of 16 states (one for each possible positions of the robot in the ice world). The robot has 4 actions available ('UP', 'RIGHT', 'DOWN' and 'LEFT'). Each action moves the robot in its direction with probability 0.95. When the robot tries to move outside of the grid, the action will have no effect with probability 1.

Slipping will happen with probability 0.05, and it results in the robot moving in the direction specified by the action up to either the grid border, or to the first crack (into it). (NB: it is possible for both slipping and not slipping as a result of an action to result in the same state, i.e., when the robot is one step away from the edge in the direction of movement, or when the agent is already at the edge and cannot move in that direction any further. Therefore, the transition function for states near the edge (for a given action/direction) looks different from states further away from the edge.)


## Q-Learning implementation

We start the implementation by importing the necessary libraries. We use [PyTorch](https://pytorch.org) to implement the Q-value table, but feel free to import [Numpy](https://numpy.org) and construct a new one if you are more comfortable with it. We import the implementation of the Ice World, described above, from [ice_env.py](../edit/ice_env.py). 

In [1]:
import random
import torch

from ice_env import Ice

Q-Learning is the off-policy Temporal Difference (TD) algorithm for optimal policy estimation. We provide the pseudo-code algorithm below, as presented by R. Sutton and A. Barto in Reinforcement Learning: An itroduction.

0. Algorithm hyper-parameters: $\alpha \in (0, 1]$ (**LEARNING_RATE** in the code), $\gamma \in (0, 1]$  (**GAMMA**), and small $\varepsilon > 0$ (**EPSILON**)
1. Initialize $Q(s,a)$, for all $S \in S^+$ and  $a \in A(s)$, arbitrarily except that $Q(\text{terminal},·) = 0$. $S^+$ here represents the set of **all** environment states, including the terminal ones. $A(s)$ represents the action set defined over each state, which is the same accross all states in our Ice World Problem (except terminal ones).
2. **for each** EPISODE **in** **EPISODES**
    1. Initialize S (restart the episode and get starting state)
    2. **while** S **is not terminal** 
        1. Choose $A$ in $S$ using policy derived from $Q(S, A)$ (in this exercise we use $\varepsilon$-greedy policy)
        2. Take action $A$, observe $R$, $S'$
        3. Update $Q(S,A)$: $Q(S,A) = Q(S,A) + \alpha[R + \max_a Q(S',a) - Q(S,A)$]
        4. Set $S'$ as current state $S$: $S \leftarrow S'$

Your task in this assignment is to implement the pseudo-code above. The output policy of a sucessfull solution, with the current set of hyper-parameters, should look like this:


<figure class="image">
  <img src="./resources/q_policy.png" alt="Ice World environment." width="200">
</figure>

Symbols **>** and **<** indicate that the agent takes actions 'RIGHT' and 'LEFT', respectively, while **^** and **v** indicate actions 'UP' and 'DOWN'. Can you guess why certain states are marked with **?** ? 

We start the implementation by setting the hyper-parameters, creating the Ice World Environment **env** and initializing the Q-value table **qtable**. We also provide the loop structure (2., and 2.B. in pseudo-code above) and leave the rest to you :) Places where the code should be filled in are marked with comments starting with *Task 1* through *Task 5*.

In [ ]:
EPISODES = 1000
# ### HYPER PARAMETERS ###############
# Exploration vs Exploitation tradeoff
EPSILON = 0.15 
# Discount factor
GAMMA = 0.9
# Learning rate 
LEARNING_RATE = 0.1
# ####################################

# The Ice World Environment implementation Ice has two methods that 
# should be of particular interest for tasks in this exercise:
# * step(action) -> next_state, reward, is_terminal
# * reset() -> start_state
# Feel free to review the file ice_env.py.
env = Ice(slip_chance=0.05, ice_r=-5.0, crack_r=-10.0, treasure_r=20.0, goal_r=100.0)
average_cumulative_reward = 0.0

# Q-table, for each env state have action space
# (current example: 4x4 states, 4 actions per state)
# Note that we do not care how these actions are ordered,
# as long as the ordering is consistent between states
qtable = torch.zeros(env.env_space() + env.action_space(), dtype=torch.float)

# Loop over episodes (pseudo-code step 2.)
for i in range(EPISODES):
    # Start state initialization (pseudo-code step 2.A)
    state = env.reset()
    is_terminal = False
    cumulative_reward = 0.0

    # Loop over time-steps (pseudo-code step 2.B.)
    while not is_terminal:
        # 0 Currently you are in "state S (state)"
        
        # Task 1: Calculate action to be taken from state S. Use 'e-rand off-policy'.
        # 1.1 Determine whether to take greedy action, based on current Q-value
        # estimates, or random action
        # 1.2 If greedy action is determined to be taken, implement the selection
        # of this action
        # OR
        # 1.3 If random action is determined to be taken, implement random action
        # selection
        
        
        # Task 2: Perform the action
        


        # Task 3: Update the q-table
        

        
        # Task 4: Update cumulative reward

        

        # Task 5: Make current state next state



    print("EPISODE: {0: <4}/{1: >4} | EPSILON: {2: <7.3f} | SCORE: {3: >7.1f}".format(i + 1, EPISODES, EPSILON, cumulative_reward))
    average_cumulative_reward += cumulative_reward

average_cumulative_reward /= EPISODES
print()
env.print_qtable_stats(qtable)

## Experimentation

The end goal of this exercise is for you to get the feel of applying RL. To that effect, try to answer to questions below:

1. Is the resulting policy optimal?
2. Why is the cumulative reward (return) fluctuating even in later stages of training? What are the sources of  stochasticity in the overall solution?
3. How does the change in hyper-parameters affect the resulting policy?
    * Try increasing and decreasing EPSILON parameter. Does the algorithm converge to optimal policy with these modified values?
    * How does changing the LEARNING_RATE affect policy convergence?
4. Can you explain the "?" in the resulting policy? Hint: You can inspect the actual Q-values of each state and action pair in the solution printout.
5. How does the policy change if we assign the value of 100 to the "Treasure state", or reduce "slip chance" to 0?